<img style="margin-left: auto; margin-right: auto; width=50%" src="uproot-3.png"></img>

What's new?

   * modularization
   * writing files
   * jagged array operations

## Modularization

uproot 2 had been a single library (optionally depending on lz4).

<img style="margin-left: auto; margin-right: auto; width=80%" src="abstraction-layers-before.png"></img>

## Modularization

uproot 3 splits out everything that is not I/O. We'll see the advantage in a moment.

<img style="margin-left: auto; margin-right: auto; width=80%" src="abstraction-layers.png"></img>